In [46]:
import pandas as pd
data =pd.read_csv('AAPL.csv')

In [256]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [258]:
data.head()

,Close,Lag_1,Lag_2,Lag_3
Date,,,,
1980-12-17,0.000365,0.000350,0.000399,0.000435
1980-12-18,0.000383,0.000365,0.000350,0.000399
1980-12-19,0.000423,0.000383,0.000365,0.000350
1980-12-22,0.000457,0.000423,0.000383,0.000365
1980-12-23,0.000488,0.000457,0.000423,0.000383


In [50]:
data.columns = data.columns.str.strip()


In [52]:
print(data.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


Convert 'Date' column to datetime format.......
date columns in datasets are stored as strings, making it difficult to perform time-based operations.



In [55]:
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data[['Close']]

In [57]:
print(data)

                 Close
Date                  
1980-12-12    0.128348
1980-12-15    0.121652
1980-12-16    0.112723
1980-12-17    0.115513
1980-12-18    0.118862
...                ...
2022-03-18  163.979996
2022-03-21  165.380005
2022-03-22  168.820007
2022-03-23  170.210007
2022-03-24  174.070007

[10409 rows x 1 columns]


In [59]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
data['Close'] = scaler.fit_transform(data[['Close']])


In [61]:
data['Close']

Date
1980-12-12    0.000435
1980-12-15    0.000399
1980-12-16    0.000350
1980-12-17    0.000365
1980-12-18    0.000383
                ...   
2022-03-18    0.900913
2022-03-21    0.908607
2022-03-22    0.927512
2022-03-23    0.935151
2022-03-24    0.956364
Name: Close, Length: 10409, dtype: float64

In [67]:
import numpy as np

def create_sequence(data,seq_length=60):
    X,y = [],[]
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])

    return np.array(X) ,np.array(y)

seq_length =60
X,y = create_sequence(data['Close'].values , seq_length)

In [69]:
##split th sequences into training and test sets
train_size = int(len(X) * 0.8)

X_train , X_test = X[:train_size] ,X[train_size:]

y_train , y_test = y[:train_size] ,y[train_size:]

In [75]:
##build a sequential lstm  layer
from tensorflow.keras.models import Sequential ##models layer by layer
from tensorflow.keras.layers import LSTM, Dense ##dense -A fully connnected layer for final predictions



In [79]:
lstm_model =Sequential()   #sequential model where layers will be added one after another.

lstm_model.add(LSTM(units=50 , return_sequences=True,input_shape=(X_train.shape[1],1))) #x_train.shape = (800,60,1)
lstm_model.add(LSTM(units=50)) ##second layer(final layer)

lstm_model.add(Dense(1))


C:\Users\sanoj\anaconda3\new\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [81]:
###OPTIMIZER AND LOSS FN
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
259/259 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - loss: 1.0872e-04
Epoch 2/10
259/259 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 2.6726e-06
Epoch 3/10
259/259 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 2.5259e-06
Epoch 4/10
259/259 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 2.3734e-06
Epoch 5/10
259/259 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 2.3789e-06
Epoch 6/10
259/259 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 1.8730e-06
Epoch 7/10
259/259 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 1.5290e-06
Epoch 8/10
259/259 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 1.7246e-06
Epoch 9/10
259/259 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 3.1969e-06
Epoch 10/10
259/259 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 1.4735e-06


In [83]:
####LAG FEATURE FOR TIME SERIES FORECASTING   PAST 3 DAYS    FROM PAST TRENDS

data['Lag_1'] = data['Close'].shift(1)
data['Lag_2'] = data['Close'].shift(2)
data['Lag_3'] = data['Close'].shift(3)
data = data.dropna()

In [85]:
data

,Close,Lag_1,Lag_2,Lag_3
Date,,,,
1980-12-17,0.000365,0.000350,0.000399,0.000435
1980-12-18,0.000383,0.000365,0.000350,0.000399
1980-12-19,0.000423,0.000383,0.000365,0.000350
1980-12-22,0.000457,0.000423,0.000383,0.000365
1980-12-23,0.000488,0.000457,0.000423,0.000383
...,...,...,...,...
2022-03-18,0.900913,0.882447,0.876787,0.852056
2022-03-21,0.908607,0.900913,0.882447,0.876787
2022-03-22,0.927512,0.908607,0.900913,0.882447


In [89]:
##for linear regression

X_lin = data[['Lag_1','Lag_2','Lag_3']]
y_lin = data['Close']

In [93]:
X_lin

,Lag_1,Lag_2,Lag_3
Date,,,
1980-12-17,0.000350,0.000399,0.000435
1980-12-18,0.000365,0.000350,0.000399
1980-12-19,0.000383,0.000365,0.000350
1980-12-22,0.000423,0.000383,0.000365
1980-12-23,0.000457,0.000423,0.000383
...,...,...,...
2022-03-18,0.882447,0.876787,0.852056
2022-03-21,0.900913,0.882447,0.876787
2022-03-22,0.908607,0.900913,0.882447


In [95]:
X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]

In [99]:
X_train.shape

(8279, 60)

In [101]:
y_train.shape

(8279,)

In [103]:
X_test.shape

(2070, 60)

In [105]:
y_test.shape

(2070,)

In [107]:
###LiNEAR REGRESSION
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()
lin_model.fit(X_train_lin, y_train_lin)


LinearRegression()

In [111]:
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1)) ##(400, 60, 1)
lstm_predictions = lstm_model.predict(X_test_lstm)

65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


In [113]:
lstm_predictions

array([[0.10830665],
       [0.10773644],
       [0.10709514],
       ...,
       [0.802181  ],
       [0.8110388 ],
       [0.8207888 ]], dtype=float32)

In [115]:
##convert to actual price
lstm_predictions =  scaler.inverse_transform(lstm_predictions)

In [121]:
#lstm_predictions

In [123]:
lin_predictions = lin_model.predict(X_test_lin)

In [125]:
lin_predictions 

array([0.09648872, 0.09712234, 0.09763815, ..., 0.90846277, 0.92793974,
       0.93500065])

In [127]:
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1, 1))  ##1d array to 2d array

In [129]:
lin_predictions

array([[ 17.60627997],
       [ 17.72157507],
       [ 17.81543111],
       ...,
       [165.35379893],
       [168.89784602],
       [170.18265545]])

In [149]:
##fix shape mismatch-operands could not be broadcast together with shapes (2070,1) (2127,1) 
##solutions :trim the larger array  using min()
 
min_length = min(len(lstm_predictions), len(lin_predictions))


In [151]:
min_length

2070

In [139]:

lstm_predictions = lstm_predictions[:min_length]

In [143]:
lstm_predictions.shape

(2070, 1)

In [147]:
lin_predictions = lin_predictions[:min_length]
lin_predictions.shape

(2070, 1)

In [161]:
#weight = 0.8 and 0.2 means, lstm model given 70% importance 
hybrid_predictions = (0.8 * lstm_predictions) + (0.2 * lin_predictions)

In [194]:
lstm_future_predictions = np.array([]).reshape(0, 1)  # Initialize as an empty 2D array

last_sequence = X[-1].reshape(1, seq_length, 1)  # Get last known sequence
for _ in range(10):
    lstm_pred = lstm_model.predict(last_sequence)[0, 0]  # Predict next value
    
    # Convert prediction to a 2D array before concatenation
    lstm_future_predictions = np.concatenate((lstm_future_predictions, np.array([[lstm_pred]])))
    
    # Reshape and update sequence for next step
    lstm_pred_reshaped = np.array([[[lstm_pred]]])  # Ensure it's 3D
    last_sequence = np.append(last_sequence[:, 1:, :], lstm_pred_reshaped, axis=1)

# Convert predictions back to the original scale
lstm_future_predictions = scaler.inverse_transform(lstm_future_predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [196]:
recent_data = data['Close'].values[-3:]  ##last 3 values

In [248]:
recent_data


array([0.92840579])

In [270]:
recent_data = data['Close'].values[-3:] ## last 3 values
lin_future_predictions = []

for _ in range(10): 
    lin_pred = lin_model.predict(recent_data.reshape(1, -1))[0]
    
    lin_future_predictions.append(lin_pred)
    
    recent_data = np.append(recent_data[1:], lin_pred)
    
lin_future_predictions = scaler.inverse_transform(np.array(lin_future_predictions).reshape(-1, 1))

In [273]:
hybrid_future_predictions = (0.8 * lstm_future_predictions) + (0.2 * lin_future_predictions)

In [275]:
hybrid_future_predictions

array([[153.31697387],
       [153.2089896 ],
       [152.8072235 ],
       [150.04337274],
       [148.1867858 ],
       [147.00780165],
       [143.86893946],
       [141.82259314],
       [140.7545516 ],
       [137.7483626 ]])

In [277]:
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=10)
predictions_df = pd.DataFrame({
    'Date': future_dates,
    'LSTM Predictions': lstm_future_predictions.flatten(),
    'Linear Regression Predictions': lin_future_predictions.flatten(),
    'Hybrid Model Predictions': hybrid_future_predictions.flatten()
})
print(predictions_df)

        Date  LSTM Predictions  Linear Regression Predictions  \
0 2022-03-25        149.400555                     168.982649   
1 2022-03-26        149.003580                     170.030627   
2 2022-03-27        147.443674                     174.261422   
3 2022-03-28        145.263219                     169.163988   
4 2022-03-29        142.772490                     169.843970   
5 2022-03-30        140.149530                     174.440889   
6 2022-03-31        137.495085                     169.364359   
7 2022-04-01        134.865455                     169.651148   
8 2022-04-02        132.291441                     174.606993   
9 2022-04-03        129.789454                     169.583996   

   Hybrid Model Predictions  
0                153.316974  
1                153.208990  
2                152.807223  
3                150.043373  
4                148.186786  
5                147.007802  
6                143.868939  
7                141.822593  
8             